# SQL Guided Practice - Chinook Database

The [Chinook Database](https://github.com/lerocha/chinook-database) represents a digital media store with tables for artists, albums, tracks, sales, customers, employees, and more. You will be performing multiple queries to extract information from this database.

Please begin by importing the necessary packages using the standard alias:
- `sqlite3`
- `pandas`

In [1]:
# Your code here
import sqlite3
import pandas as pd

The database is stored in the `Chinook_Sqlite.sqlite` file *within* the `data` directory. In the cell below, please replace `None` with the appropriate code to establish a connection to sqlite database.

In [4]:
!ls

README.md                          sql-guided-practice-solution.ipynb
data                               sql-guided-practice.ipynb
erd.png


In [5]:
conn = sqlite3.connect('./data/Chinook_Sqlite.sqlite')

## Database Schema

There are 11 total tables in the database. The diagram below illustrates the relationships between each table.

![](erd.png)

### 1. Query the Name of every track and its associated size in Bytes. Order the results in descending order by Bytes.

In [6]:
q = """
SELECT Name, Bytes
FROM Track
ORDER BY Bytes DESC;
"""

pd.read_sql(q, conn)

,Name,Bytes
0,Through a Looking Glass,1059546140
1,Occupation / Precipice,1054423946
2,The Young Lords,587051735
3,The Man With Nine Lives,577829804
4,Dave,574325829
...,...,...
3498,Commercial 1,319888
3499,Oprah,224313
3500,A Statistic,211997
3501,Now Sports,161266


### 2. Find the total number of tracks in each playlist. The results should show the PlaylistId and total number of tracks in the playlist. As a bonus, show the Playlist name instead of the PlaylistId.

In [8]:
q = """
SELECT PlaylistId, COUNT(TrackId) AS num_tracks
FROM PlaylistTrack
GROUP BY PlaylistId;
"""

pd.read_sql(q, conn)

,PlaylistId,num_tracks
0,1,3290
1,3,213
2,5,1477
3,8,3290
4,9,1
5,10,213
6,11,39
7,12,75
8,13,25
9,14,25


In [11]:
q = """
SELECT Name, COUNT(TrackId) AS num_tracks
FROM PlaylistTrack
JOIN Playlist
    USING(PlaylistId)
GROUP BY PlaylistId;
"""

pd.read_sql(q, conn)

,Name,num_tracks
0,Music,3290
1,TV Shows,213
2,90’s Music,1477
3,Music,3290
4,Music Videos,1
5,TV Shows,213
6,Brazilian Music,39
7,Classical,75
8,Classical 101 - Deep Cuts,25
9,Classical 101 - Next Steps,25


### 3. Show the 5 countries with the most number of customers.

In [14]:
q = """
SELECT Country, COUNT(CustomerId) AS num_customers
FROM Customer
GROUP BY Country
ORDER BY num_customers DESC
LIMIT 5;
"""

pd.read_sql(q, conn)

,Country,num_customers
0,USA,13
1,Canada,8
2,France,5
3,Brazil,5
4,Germany,4


### 4. Query each invoice line item (InvoiceLineId) and include the name of the Track purchased and name of the Artist.

In [15]:
q = """
SELECT InvoiceLineId, t.Name, a.Name
FROM InvoiceLine
JOIN Track AS t
    USING(TrackId)
JOIN Album 
    USING(AlbumId)
JOIN Artist AS a
    USING(ArtistId);
"""

pd.read_sql(q, conn)

,InvoiceLineId,Name,Name
0,579,For Those About To Rock (We Salute You),AC/DC
1,1,Balls to the Wall,Accept
2,1154,Balls to the Wall,Accept
3,1728,Fast As a Shark,Accept
4,2,Restless and Wild,Accept
...,...,...,...
2235,1726,"Symphony No. 2, Op. 16 - ""The Four Temperamen...",Göteborgs Symfoniker & Neeme Järvi
2236,577,"Étude 1, In C Major - Preludio (Presto) - Liszt",Michele Campanella
2237,1153,Pini Di Roma (Pinien Von Rom) \ I Pini Della V...,Eugene Ormandy
2238,578,"String Quartet No. 12 in C Minor, D. 703 ""Quar...",Emerson String Quartet


### 5. Show average revenue per sale for each sales agent (total sales / number of sales). Round to 2 decimal places.

In [19]:
q = """
SELECT e.FirstName, e.LastName, ROUND(AVG(i.Total), 2) AS avg_revenue_per_sale
FROM Employee AS e
JOIN Customer AS c
    ON e.EmployeeId = c.SupportRepId
JOIN Invoice AS i
    USING(CustomerId)
GROUP BY e.EmployeeId;
"""

pd.read_sql(q, conn)

,FirstName,LastName,avg_revenue_per_sale
0,Jane,Peacock,5.71
1,Margaret,Park,5.54
2,Steve,Johnson,5.72


### 6. Show all customers who are NOT from one of the 5 countries with the most number of customers. Do this WITHOUT hardcoding the country names into the query. (Hint: Use a subquery)

In [49]:
q = """
WITH top_5_countries AS (SELECT Country
FROM Customer
GROUP BY Country
ORDER BY COUNT(CustomerId) DESC
LIMIT 5)

SELECT CustomerId, FirstName, LastName, Country
FROM Customer
WHERE Country NOT IN top_5_countries;
"""

pd.read_sql(q, conn)

,CustomerId,FirstName,LastName,Country
0,4,Bjørn,Hansen,Norway
1,5,František,Wichterlová,Czech Republic
2,6,Helena,Holý,Czech Republic
3,7,Astrid,Gruber,Austria
4,8,Daan,Peeters,Belgium
5,9,Kara,Nielsen,Denmark
6,34,João,Fernandes,Portugal
7,35,Madalena,Sampaio,Portugal
8,44,Terhi,Hämäläinen,Finland
9,45,Ladislav,Kovács,Hungary


In [47]:
q = """
SELECT Country
FROM Customer
GROUP BY Country
ORDER BY COUNT(CustomerId) DESC
LIMIT 5;"""

pd.read_sql(q, conn)

,Country
0,USA
1,Canada
2,France
3,Brazil
4,Germany


### 7. Find which employee sold the highest number of unique media types?

In [50]:
q = """
SELECT e.EmployeeId, e.FirstName, e.LastName, COUNT(DISTINCT m.MediaTypeId) AS unique_media_types
FROM Employee AS e
JOIN Customer AS c
    ON e.EmployeeId = c.SupportRepId
JOIN Invoice
    USING(CustomerId)
JOIN InvoiceLine
    USING(InvoiceId)
JOIN Track
    USING(TrackId)
JOIN MediaType AS m
    USING(MediaTypeId)
GROUP BY e.EmployeeId
ORDER BY unique_media_types DESC
LIMIT 1;
"""

pd.read_sql(q, conn)

,EmployeeId,FirstName,LastName,unique_media_types
0,4,Margaret,Park,5


### 8. How many Classical tracks are available in each MediaType?

In [24]:
q = """
SELECT DISTINCT Name FROM MediaType;
"""

pd.read_sql(q, conn)

,Name
0,MPEG audio file
1,Protected AAC audio file
2,Protected MPEG-4 video file
3,Purchased AAC audio file
4,AAC audio file


In [29]:
q = """
SELECT m.Name AS MediaType, COUNT(t.TrackId) AS num_tracks 
FROM Track AS t
JOIN MediaType AS m
    USING(MediaTypeId)
JOIN Genre AS g
    USING(GenreId)
WHERE g.Name == "Classical"
GROUP BY MediaType;
"""

pd.read_sql(q, conn)

,MediaType,num_tracks
0,AAC audio file,1
1,Protected AAC audio file,67
2,Purchased AAC audio file,6


In [31]:
q = """
SELECT m.Name AS MediaType, COUNT(t.TrackId) AS num_tracks
FROM MediaType AS m
JOIN Track AS t
    USING(MediaTypeId)
JOIN Genre AS g
    USING(GenreId)
WHERE g.Name = "Classical"
GROUP BY MediaType;
"""

pd.read_sql(q, conn)

,MediaType,num_tracks
0,AAC audio file,1
1,Protected AAC audio file,67
2,Purchased AAC audio file,6


### 9. What are the top 5 most successful (profitable) Rock artists of all time?

In [42]:
q = """
SELECT a.ArtistId, a.Name, SUM(i.Total) AS total_sales
FROM Artist AS a
JOIN Album
    USING(ArtistId)
JOIN Track
    USING(AlbumId)
JOIN InvoiceLine
    USING(TrackId)
JOIN Invoice AS i
    USING(InvoiceId)
JOIN Genre AS g
    USING(GenreId)
WHERE g.Name == 'Rock'
GROUP BY a.ArtistId
ORDER BY total_sales DESC
LIMIT 5;
"""

pd.read_sql(q, conn)

,ArtistId,Name,total_sales
0,150,U2,773.65
1,22,Led Zeppelin,620.73
2,58,Deep Purple,550.44
3,90,Iron Maiden,473.22
4,152,Van Halen,336.82


### 10. Which 3 customers have spent the most on Jazz music in MPEG format?

In [51]:
q = """
WITH jazz_mpeg_track_ids AS (SELECT t.TrackId
FROM Track AS t
WHERE t.GenreId IN (
SELECT GenreId
FROM Genre
WHERE Name == 'Jazz')
AND t.MediaTypeId IN (
SELECT MediaTypeId
FROM MediaType
WHERE Name LIKE '%MPEG%'))

SELECT c.FirstName, c.LastName, SUM(i.Total) AS total_spent
FROM Customer AS c
JOIN Invoice AS i
    USING(CustomerId)
JOIN InvoiceLine AS il
    USING(InvoiceId)
WHERE il.TrackId IN jazz_mpeg_track_ids
GROUP BY c.CustomerId
ORDER BY total_spent DESC
LIMIT 3;
"""

pd.read_sql(q, conn)

,FirstName,LastName,total_spent
0,Dominique,Lefebvre,83.16
1,Puja,Srivastava,69.30
2,Dan,Miller,55.44
